In [1]:
#Create a data frame from a JSON file
empDf = sqlContext.read.json("data/customerData.json")
empDf.show()

+---+------+------+-----------------+------+
|age|deptid|gender|             name|salary|
+---+------+------+-----------------+------+
| 32|   100|  male|Benjamin Garrison|  3000|
| 40|   200|  male|    Holland Drake|  4500|
| 26|   100|  male|  Burks Velasquez|  2700|
| 51|   100|female|    June Rutledge|  4300|
| 44|   200|  male|    Nielsen Knapp|  6500|
+---+------+------+-----------------+------+



In [2]:
empDf.printSchema()

root
 |-- age: string (nullable = true)
 |-- deptid: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- name: string (nullable = true)
 |-- salary: string (nullable = true)



In [3]:
#Do SQL queries
empDf.select("name").show()

+-----------------+
|             name|
+-----------------+
|Benjamin Garrison|
|    Holland Drake|
|  Burks Velasquez|
|    June Rutledge|
|    Nielsen Knapp|
+-----------------+



In [4]:
empDf.filter(empDf["age"] == 40).show()

+---+------+------+-------------+------+
|age|deptid|gender|         name|salary|
+---+------+------+-------------+------+
| 40|   200|  male|Holland Drake|  4500|
+---+------+------+-------------+------+



In [5]:
empDf.groupBy("gender").count().show()

+------+-----+
|gender|count|
+------+-----+
|female|    1|
|  male|    4|
+------+-----+



In [6]:
empDf.groupBy("deptid").agg({"salary": "avg", "age": "max"}).show()

+------+------------------+--------+
|deptid|       avg(salary)|max(age)|
+------+------------------+--------+
|   200|            5500.0|      44|
|   100|3333.3333333333335|      51|
+------+------------------+--------+



In [7]:
#create a data frame from a list
deptList = [{'name': 'Sales', 'id': "100"},{ 'name':'Engineering','id':"200" }]
deptDf = sqlContext.createDataFrame(deptList)
deptDf.show()

/home/isaac/spark/spark-1.6.3-bin-hadoop2.6/python/pyspark/sql/context.py:239: UserWarning: inferring schema from dict is deprecated,please use pyspark.sql.Row instead
  warnings.warn("inferring schema from dict is deprecated,"


+---+-----------+
| id|       name|
+---+-----------+
|100|      Sales|
|200|Engineering|
+---+-----------+



In [8]:
#join the data frames
empDf.join(deptDf, empDf.deptid == deptDf.id).show()

+---+------+------+-----------------+------+---+-----------+
|age|deptid|gender|             name|salary| id|       name|
+---+------+------+-----------------+------+---+-----------+
| 32|   100|  male|Benjamin Garrison|  3000|100|      Sales|
| 26|   100|  male|  Burks Velasquez|  2700|100|      Sales|
| 51|   100|female|    June Rutledge|  4300|100|      Sales|
| 40|   200|  male|    Holland Drake|  4500|200|Engineering|
| 44|   200|  male|    Nielsen Knapp|  6500|200|Engineering|
+---+------+------+-----------------+------+---+-----------+



In [9]:
#cascading operations
empDf.filter(empDf["age"] >30).join(deptDf, \
        empDf.deptid == deptDf.id).\
        groupBy("deptid").\
        agg({"salary": "avg", "age": "max"}).show()

+------+-----------+--------+
|deptid|avg(salary)|max(age)|
+------+-----------+--------+
|   200|     5500.0|      44|
|   100|     3650.0|      51|
+------+-----------+--------+



In [10]:
#register a data frame as table and run SQL statements against it
empDf.registerTempTable("employees")
sqlContext.sql("select * from employees where salary > 4000").show()

+---+------+------+-------------+------+
|age|deptid|gender|         name|salary|
+---+------+------+-------------+------+
| 40|   200|  male|Holland Drake|  4500|
| 51|   100|female|June Rutledge|  4300|
| 44|   200|  male|Nielsen Knapp|  6500|
+---+------+------+-------------+------+



In [11]:
#to pandas data frame
empPands = empDf.toPandas()
for index, row in empPands.iterrows():
    print row["salary"]

3000
4500
2700
4300
6500


In [12]:
#............................................................................
##   Working with Databases
#............................................................................
#Make sure that the spark classpaths are set appropriately in the 
#spark-defaults.conf file to include the driver files
    
demoDf = sqlContext.read.format("jdbc").options(
    url="jdbc:mysql://localhost:3306/demo",
    driver = "com.mysql.jdbc.Driver",
    dbtable = "demotable",
    user="root",
    password="").load()

Py4JJavaError: An error occurred while calling o77.load.
: java.lang.ClassNotFoundException: com.mysql.jdbc.Driver
	at java.net.URLClassLoader.findClass(URLClassLoader.java:381)
	at java.lang.ClassLoader.loadClass(ClassLoader.java:424)
	at java.lang.ClassLoader.loadClass(ClassLoader.java:357)
	at org.apache.spark.sql.execution.datasources.jdbc.DriverRegistry$.register(DriverRegistry.scala:38)
	at org.apache.spark.sql.execution.datasources.jdbc.JdbcUtils$$anonfun$createConnectionFactory$1.apply(JdbcUtils.scala:45)
	at org.apache.spark.sql.execution.datasources.jdbc.JdbcUtils$$anonfun$createConnectionFactory$1.apply(JdbcUtils.scala:45)
	at scala.Option.foreach(Option.scala:236)
	at org.apache.spark.sql.execution.datasources.jdbc.JdbcUtils$.createConnectionFactory(JdbcUtils.scala:45)
	at org.apache.spark.sql.execution.datasources.jdbc.JDBCRDD$.resolveTable(JDBCRDD.scala:120)
	at org.apache.spark.sql.execution.datasources.jdbc.JDBCRelation.<init>(JDBCRelation.scala:91)
	at org.apache.spark

In [13]:
demoDf.show()

NameError: name 'demoDf' is not defined

In [15]:
from pyspark.sql import Row
lines = sc.textFile("data/auto-data.csv")
#remove the first line
datalines = lines.filter(lambda x: "FUELTYPE" not in x)
datalines.count()

197

In [16]:
datalines = lines.filter(lambda x: "FUELTYPE" not in x)
datalines.count()

197

In [17]:
parts = datalines.map(lambda l: l.split(","))
autoMap = parts.map(lambda p: Row(make=p[0],\
         body=p[4], hp=int(p[7])))
autoMap.collect()

[Row(body=u'hatchback', hp=69, make=u'subaru'),
 Row(body=u'hatchback', hp=48, make=u'chevrolet'),
 Row(body=u'hatchback', hp=68, make=u'mazda'),
 Row(body=u'hatchback', hp=62, make=u'toyota'),
 Row(body=u'hatchback', hp=68, make=u'mitsubishi'),
 Row(body=u'hatchback', hp=60, make=u'honda'),
 Row(body=u'sedan', hp=69, make=u'nissan'),
 Row(body=u'hatchback', hp=68, make=u'dodge'),
 Row(body=u'hatchback', hp=68, make=u'plymouth'),
 Row(body=u'hatchback', hp=68, make=u'mazda'),
 Row(body=u'hatchback', hp=68, make=u'mitsubishi'),
 Row(body=u'hatchback', hp=68, make=u'dodge'),
 Row(body=u'hatchback', hp=68, make=u'plymouth'),
 Row(body=u'hatchback', hp=70, make=u'chevrolet'),
 Row(body=u'hatchback', hp=62, make=u'toyota'),
 Row(body=u'hatchback', hp=68, make=u'dodge'),
 Row(body=u'hatchback', hp=58, make=u'honda'),
 Row(body=u'hatchback', hp=62, make=u'toyota'),
 Row(body=u'hatchback', hp=76, make=u'honda'),
 Row(body=u'sedan', hp=70, make=u'chevrolet'),
 Row(body=u'sedan', hp=69, make=u'n

In [18]:
# Infer the schema, and register the DataFrame as a table.
autoDf = sqlContext.createDataFrame(autoMap)
autoDf.registerTempTable("autos")
sqlContext.sql("select * from autos where hp > 200").show()

+-----------+---+-------+
|       body| hp|   make|
+-----------+---+-------+
|    hardtop|207|porsche|
|    hardtop|207|porsche|
|      sedan|262| jaguar|
|convertible|207|porsche|
+-----------+---+-------+

